# Import package

In [1]:
from datetime import datetime
import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.pyplot import MultipleLocator
%matplotlib inline
import numpy as np
from numpy import mean
from numpy import std
from numpy import sort
import pandas as pd
from pandas.core.frame import DataFrame
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score  
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score,accuracy_score, plot_confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import log_loss
import seaborn as sns
from time import time
import pprint
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper,DeltaYStopper
from skopt.space import Real,Categorical,Integer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
import joblib
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

# Dataset

In [2]:
arrhyData=pd.read_csv('Arrhythmia cleaned data.csv')
arrhyData.head()

,Age,Sex,Height,Weight,QRS duration,P-R interval,Q-T interval,T interval,P interval,QRS,...,Q wave V6,R wave V6,S wave V6,R' wave V6,S' wave V6,P wave V6,T wave V6,QRSA V6,QRSTA V6,label
0,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,5
1,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,7
2,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,0
3,40,1,160,52,77,129,377,133,77,77,...,0.0,6.5,0.0,0.0,0.0,0.4,1.0,14.3,20.5,0
4,49,1,162,54,78,0,376,157,70,67,...,0.0,8.2,-1.9,0.0,0.0,0.1,0.5,15.8,19.8,0


In [3]:
arrhyData01=arrhyData.copy()

In [4]:
X=arrhyData01.drop(labels=['label'],axis=1)

y=arrhyData01['label'].values

In [5]:
#stanardilization

scaler=StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

In [6]:
pca1 =PCA(n_components=None, copy=True, whiten=False)
pcamodel1 = pca1.fit(X)

In [7]:
print("Explained Variance: ") 
print(pcamodel1.components_)

pcamodel1.n_components_

newMat1=pcamodel1.fit_transform(X) 
X_pca1 = DataFrame(newMat1)
X31_train, X31_test,y31_train,y31_test= train_test_split(X_pca1,y,test_size=0.2,shuffle=True,stratify=y,random_state=42)

Explained Variance: 
[[ 6.73351726e-02 -1.90862962e-02  1.72037300e-03 ... -9.14346524e-02
  -8.66927095e-02 -1.30482728e-01]
 [ 5.88844957e-02  8.46249319e-03 -3.67779384e-02 ... -8.23138154e-02
   1.85551382e-01  6.68947820e-02]
 [-1.02524819e-02 -4.55955088e-02  2.09373517e-02 ... -6.23684539e-02
  -1.86015633e-02 -6.03188860e-02]
 ...
 [ 0.00000000e+00  1.45728434e-17  2.12146261e-16 ... -1.73472348e-17
  -3.37403716e-16  1.79977561e-17]
 [ 0.00000000e+00 -1.74153893e-31 -3.80182235e-31 ... -4.03674916e-31
  -8.24298016e-32  4.16963833e-31]
 [-0.00000000e+00 -1.35134848e-31 -9.38973165e-32 ... -3.29719206e-31
  -2.08867102e-31  4.00978614e-31]]


In [8]:
classifier_lgbm_pca=LGBMClassifier()

classifier_svm_pca=SVC(kernel='rbf',gamma=0.001,decision_function_shape='ovr',C=100,degree=1,probability=True)
classifier_rf_pca= RFC(random_state=42, max_features='log2', n_estimators= 150,
                                     max_depth=11, 
                                     criterion='gini')

start=datetime.now()
classifier_lgbm_pca.fit(X31_train,y31_train.ravel())
stop=datetime.now()
execution_time_svm=stop-start
print("LGBM execution time is:",execution_time_svm)


start=datetime.now()
classifier_svm_pca.fit(X31_train,y31_train.ravel())
stop=datetime.now()
execution_time_svm=stop-start
print("SVM execution time is:",execution_time_svm)

classifier_rf_pca.fit(X31_train,y31_train.ravel())

#prediction on test data
y_pred_lgbm=classifier_lgbm_pca.predict(X31_test)
y_pred_svm=classifier_svm_pca.predict(X31_test)
y_pred_rf=classifier_rf_pca.predict(X31_test)
# y2_pred_svm=[list(x).index(max(x))for x in y2_pred_svm]

print(classification_report(y31_test, y_pred_lgbm,zero_division=1))
print(classification_report(y31_test, y_pred_svm,zero_division=1))
print(classification_report(y31_test, y_pred_rf,zero_division=1))

LGBM execution time is: 0:00:01.405229
SVM execution time is: 0:00:00.111081
              precision    recall  f1-score   support

           0       0.67      0.98      0.79        49
           1       1.00      0.33      0.50         9
           2       1.00      0.67      0.80         3
           3       0.67      0.67      0.67         3
           4       1.00      0.00      0.00         3
           5       1.00      0.00      0.00         5
           6       1.00      1.00      1.00         2
           7       0.83      0.50      0.62        10
           8       1.00      0.00      0.00         4

    accuracy                           0.70        88
   macro avg       0.91      0.46      0.49        88
weighted avg       0.78      0.70      0.64        88

              precision    recall  f1-score   support

           0       0.78      0.86      0.82        49
           1       0.42      0.56      0.48         9
           2       1.00      1.00      1.00         3
 

## PCA----SVM

In [9]:
X_train, X_test, y_train, y_test= train_test_split(X,y, train_size = 0.8,stratify=y,random_state=42)

In [10]:
svcpca = PCA(n_components=0.7429)

svcpca.fit(X_train)    
  
X_train_pca_svc = svcpca.transform(X_train)
X_test_pca_svc = svcpca.transform(X_test)   

In [11]:
classifier_svm_pca1=SVC(random_state=42
           ,C= 1
           ,kernel='poly'
           ,degree= 1
          ,gamma=0.01
          , probability=True)
# SVC(kernel='rbf',gamma=0.001,decision_function_shape='ovr',C=100,degree=1,probability=True)

In [12]:
classifier_svm_pca1.fit(X_train_pca_svc,y_train)
y_predict_svc = classifier_svm_pca1.predict(X_test_pca_svc)
classifier_svm_pca1.score(X_test_pca_svc, y_test) 

0.7045454545454546

In [13]:
start=datetime.now()
pca_svm_arrhthymia=classifier_svm_pca1.fit(X_train_pca_svc,y_train)
stop=datetime.now()
pca_training_time_svm='%.4f'%(stop-start).total_seconds()
print(pca_training_time_svm)

0.0365


In [14]:
y_predict_svm = classifier_svm_pca1.predict(X_test_pca_svc)

In [15]:
y_prob_svm=classifier_svm_pca1.predict_proba(X_test_pca_svc)

In [16]:
pca_svm_accuracy1='%.4f'%accuracy_score(y_test, y_predict_svm)
pca_svm_f11='%.4f'%f1_score(y_test, y_predict_svm,average='micro')
pca_svm_roc_auc1='%.4f'% roc_auc_score(y_test,y_prob_svm,multi_class='ovo',labels=np.unique(y_test))
pca_svm_loss1='%.4f'%log_loss(y_test,y_prob_svm)
print(pca_svm_accuracy1)
print(pca_svm_f11)
print(pca_svm_roc_auc1)
print(pca_svm_loss1)

0.7045
0.7045
0.7999
1.0093


In [17]:
joblib.dump(pca_svm_arrhthymia,'pca_svm_arrhthymia.model')

['pca_svm_arrhthymia.model']

In [18]:
rfpca_svm_arrhthymia=joblib.load('pca_svm_arrhthymia.model')
   #应用模型进行预测
rfresultsvm=rfpca_svm_arrhthymia.score(X_test_pca_svc,y_test)
rfresultsvm

0.7045454545454546

##  PCA----Light GBM

In [19]:
clf_lgbmtest = LGBMClassifier()

start=datetime.now()
pca_lgbm_arrhthymia=clf_lgbmtest.fit(X31_train, y31_train)
stop=datetime.now()
pca_training_time_lgbm='%.4f'%(stop-start).total_seconds()
print(pca_training_time_lgbm)

1.3576


In [20]:
y_predict_lgbm = clf_lgbmtest.predict(X31_test)

In [21]:
y_prob_lgbm = clf_lgbmtest.predict_proba(X31_test)

In [22]:
clf_lgbmtest.score(X31_test,y31_test)

0.7045454545454546

In [23]:
pca_lgbm_accuracy='%.4f'%accuracy_score(y31_test, y_predict_lgbm)
pca_lgbm_f1='%.4f'%f1_score(y31_test, y_predict_lgbm,average='micro')
pca_lgbm_roc_auc='%.4f'% roc_auc_score(y31_test,y_prob_lgbm,multi_class='ovr',labels=np.unique(y31_test))
pca_lgbm_loss='%.4f'%log_loss(y31_test,y_prob_lgbm)
print(pca_lgbm_accuracy)
print(pca_lgbm_f1)
print(pca_lgbm_roc_auc)
print(pca_lgbm_loss)

0.7045
0.7045
0.7685
1.9033


In [24]:
joblib.dump(pca_lgbm_arrhthymia,'pca_lgbm_arrhthymia.model')

['pca_lgbm_arrhthymia.model']

In [25]:
rfpca_lgbm_arrhthymia=joblib.load('pca_lgbm_arrhthymia.model')
  
rfresultlgbm=rfpca_lgbm_arrhthymia.score(X31_test,y31_test)
rfresultlgbm

0.7045454545454546

##  PCA----RF

In [26]:
rfpca = PCA(n_components=0.4786)

rfpca.fit(X_train)    
  
X_train_pca_rf = rfpca.transform(X_train)
X_test_pca_rf = rfpca.transform(X_test)   

In [27]:
classifier_rf_pca1=SVC(kernel='rbf',gamma=0.001,decision_function_shape='ovr',C=100,degree=1,probability=True)

In [28]:
classifier_rf_pca1.fit(X_train_pca_rf,y_train)
y_predict_rf = classifier_rf_pca1.predict(X_test_pca_rf)
classifier_rf_pca1.score(X_test_pca_rf, y_test) 

0.6704545454545454

In [29]:
start=datetime.now()
pca_rf_arrhthymia=classifier_rf_pca1.fit(X_train_pca_rf,y_train)
stop=datetime.now()
pca_training_time_rf='%.4f'%(stop-start).total_seconds()
print(pca_training_time_rf)

0.0582


In [30]:
classifier_rf_pca1.score(X_test_pca_rf,y_test)
y_predict_rf=classifier_rf_pca1.predict(X_test_pca_rf)
y_prob_rf=classifier_rf_pca1.predict_proba(X_test_pca_rf)


In [31]:
pca_rf_accuracy1='%.4f'%accuracy_score(y_test, y_predict_rf)
pca_rf_f11='%.4f'%f1_score(y_test, y_predict_rf,average='micro')
pca_rf_roc_auc1='%.4f'% roc_auc_score(y_test,y_prob_rf,multi_class='ovr',labels=np.unique(y31_test))
pca_rf_loss1='%.4f'%log_loss(y_test,y_prob_rf)
print(pca_rf_accuracy1)
print(pca_rf_f11)
print(pca_rf_roc_auc1)
print(pca_rf_loss1)

0.6705
0.6705
0.8243
1.0626


In [32]:
joblib.dump(pca_rf_arrhthymia,'pca_rf_arrhthymia.model')

['pca_rf_arrhthymia.model']

In [33]:
rfpca_rf_arrhthymia=joblib.load('pca_rf_arrhthymia.model')
   #应用模型进行预测
rfresultrf=rfpca_rf_arrhthymia.score(X_test_pca_rf,y_test)

In [34]:
rfresultrf

0.6704545454545454

## PCA--ANN

In [35]:
X_train = torch.from_numpy(X31_train.values)
y_train = torch.from_numpy(y31_train ).type(torch.LongTensor)

X_test  = torch.from_numpy(X31_test.values )
y_test= torch.from_numpy(y31_test).type(torch.LongTensor)

batch_size = 128
train = torch.utils.data.TensorDataset(X_train, y_train )
test = torch.utils.data.TensorDataset(X_test , y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

In [36]:
class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1,hidden_dim2, output_dim):
        super(ANNModel, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU() 
        
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [39]:
epochs =30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_dim = X_pca1.shape[1]
hidden_dim1 = 55 #hidden layer1
hidden_dim2 = 123 #hidden layer2 
output_dim = len(set(y))

In [40]:
model = ANNModel(input_dim, hidden_dim1,hidden_dim2,output_dim)


learning_rate = 0.0067
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()


In [41]:
start=datetime.now()
total_step = len(train_loader)
model.train()
for epoch in range(epochs):
    tot_loss = 0.0
    tot_acc = 0.0
    train_preds = []
    train_trues = []
  # model.train()
    for i,(train_data_batch, train_label_batch) in enumerate(train_loader):
        train_data_batch = train_data_batch.float().to(device) # 将double数据转换为float
        train_label_batch = train_label_batch.to(device)
        outputs = model(train_data_batch)
        # _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, train_label_batch)
        # print(loss)
       
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
   
        tot_loss += loss.data
        train_outputs = outputs.argmax(dim=1)
        train_preds.extend(train_outputs.detach().cpu().numpy())
        train_trues.extend(train_label_batch.detach().cpu().numpy())
        # tot_acc += (outputs.argmax(dim=1) == train_label_batch).sum().item()
        sklearn_accuracy = accuracy_score(train_trues, train_preds)
        sklearn_precision = precision_score(train_trues, train_preds, average='micro')
        sklearn_recall = recall_score(train_trues, train_preds, average='micro')
        sklearn_f1 = f1_score(train_trues, train_preds, average='micro')
        print("[sklearn_metrics] Epoch:{} loss:{:.4f} accuracy:{:.4f} precision:{:.4f} recall:{:.4f} f1:{:.4f}".format(epoch, tot_loss, sklearn_accuracy, sklearn_precision, sklearn_recall, sklearn_f1))
stop=datetime.now()
execution_time_ann=(stop-start)
pca_training_time_ann='%.4f'%(execution_time_ann).total_seconds()
print(pca_training_time_ann)
        
test_preds = []
test_trues = []
model.eval()
with torch.no_grad():
    for i,(test_data_batch, test_data_label) in enumerate(test_loader):
        test_data_batch = test_data_batch.float().to(device) # 将double数据转换为float
        test_data_label = test_data_label.to(device)
        test_outputs = model(test_data_batch)
        probs = F.softmax(test_outputs, dim=1) 
        test_outputs = test_outputs.argmax(dim=1)
        testloss = criterion(probs, test_data_label)
#         preds = torch.argmax(logits, dim=1)
        test_preds.extend(test_outputs.detach().cpu().numpy())
        test_trues.extend(test_data_label.detach().cpu().numpy())
        sklearn_accuracy = accuracy_score(test_trues, test_preds)
        sklearn_precision = precision_score(test_trues, test_preds, average='micro')
        sklearn_recall = recall_score(test_trues, test_preds, average='micro')
        sklearn_f1 = f1_score(test_trues, test_preds, average='micro')
        print(classification_report(test_trues, test_preds))
        conf_matrix = confusion_matrix(test_trues, test_preds)
        print(conf_matrix)
#         plot_confusion_matrix(conf_matrix)
        print("[sklearn_metrics] accuracy:{:.4f} precision:{:.4f} recall:{:.4f} f1:{:.4f}".format(sklearn_accuracy, sklearn_precision, sklearn_recall, sklearn_f1))

[sklearn_metrics] Epoch:0 loss:2.1876 accuracy:0.1094 precision:0.1094 recall:0.1094 f1:0.1094
[sklearn_metrics] Epoch:0 loss:4.2202 accuracy:0.3359 precision:0.3359 recall:0.3359 f1:0.3359
[sklearn_metrics] Epoch:0 loss:6.0919 accuracy:0.4000 precision:0.4000 recall:0.4000 f1:0.4000
[sklearn_metrics] Epoch:1 loss:1.6533 accuracy:0.5547 precision:0.5547 recall:0.5547 f1:0.5547
[sklearn_metrics] Epoch:1 loss:3.2069 accuracy:0.5547 precision:0.5547 recall:0.5547 f1:0.5547
[sklearn_metrics] Epoch:1 loss:4.6045 accuracy:0.5600 precision:0.5600 recall:0.5600 f1:0.5600
[sklearn_metrics] Epoch:2 loss:1.2928 accuracy:0.5469 precision:0.5469 recall:0.5469 f1:0.5469
[sklearn_metrics] Epoch:2 loss:2.3889 accuracy:0.5547 precision:0.5547 recall:0.5547 f1:0.5547
[sklearn_metrics] Epoch:2 loss:3.5661 accuracy:0.5629 precision:0.5629 recall:0.5629 f1:0.5629
[sklearn_metrics] Epoch:3 loss:0.9881 accuracy:0.6953 precision:0.6953 recall:0.6953 f1:0.6953
[sklearn_metrics] Epoch:3 loss:1.8659 accuracy:0.7

In [42]:
pca_ann_loss='%.4f'% testloss

In [43]:
pca_ann_f1='%.4f'% f1_score(test_trues, test_preds, average='micro')

In [44]:
pca_ann_accuracy='%.4f'% accuracy_score(test_trues, test_preds)

In [45]:
pca_ann_roc_auc='%.4f'% roc_auc_score(test_data_label,probs,multi_class='ovr',labels=np.unique(test_data_label))


In [46]:
PCA_dict = { 
              'accuracy':[pca_lgbm_accuracy
                          ,pca_svm_accuracy1,pca_rf_accuracy1,pca_ann_accuracy],
               'f1_score': [pca_lgbm_f1
                            ,pca_svm_f11,pca_rf_f11,pca_ann_f1],
               'roc_auc': [pca_lgbm_roc_auc
                           ,pca_svm_roc_auc1,pca_rf_roc_auc1,pca_ann_roc_auc],
               'cross_entropy_loss': [pca_lgbm_loss
                                      ,pca_svm_loss1,pca_rf_loss1,pca_ann_loss],
                'training_time[s]':[pca_training_time_lgbm
                                    ,pca_training_time_svm,pca_training_time_rf,pca_training_time_ann]
               }
pr_df_pca = pd.DataFrame(PCA_dict,index=['Light GBM','SVM','Random Forest','PyTorch ANN'])
pr_df_pca

,accuracy,f1_score,roc_auc,cross_entropy_loss,training_time[s]
Light GBM,0.7045,0.7045,0.7685,1.9033,1.3576
SVM,0.7045,0.7045,0.7999,1.0093,0.0365
Random Forest,0.6705,0.6705,0.8243,1.0626,0.0582
PyTorch ANN,0.7273,0.7273,0.8239,1.6606,0.5180


In [47]:
pr_df_pca.to_csv('Arrhthymia PCA Score.csv')